In [2]:
import numpy as np 
import plotly.graph_objects as go
from ipywidgets import interact,widgets

In [3]:
def update_layout_of_graph(fig: go.Figure,title: str = 'Plot')->go.Figure:
    fig.update_layout(
        width=800,
        height=600,
        autosize=False,
        plot_bgcolor='rgba(0,0,0,0)',
        title=title,
        
    )
    fig.update_layout(plot_bgcolor='rgba(0,0,0,0)',
                      xaxis_title = 'input values',
                      yaxis_title = 'output values',
                      legend=dict(yanchor="top",
                                  y=0.9,
                                  xanchor="right",
                                  x=0.95),
                      title={
                          'x': 0.5,
                          'xanchor': 'center'
                      })
    fig.update_xaxes(showline=True, linewidth=1, linecolor='black')
    fig.update_yaxes(showline=True, linewidth=1, linecolor='black')
    return fig

In [4]:
def line_scatter(
    visible: bool = True,
    x_lines: np.array = np.array([]),
    y_lines: np.array = np.array([]),
    name_line: str = 'Predicted function',
    showlegend: bool = True,
) -> go.Scatter:
    # Adding the lines
    return go.Scatter(
        visible=visible,
        line=dict(color="blue", width=2),
        x=x_lines,
        y=y_lines,
        name=name_line,
        showlegend= showlegend
    )

In [5]:
def dot_scatter(
    visible: bool = True,
    x_dots: np.array = np.array([]),
    y_dots: np.array = np.array([]),
    name_dots: str = 'Observed points',
    showlegend: bool = True
) -> go.Scatter:
    # Adding the dots
    return go.Scatter(
        x=x_dots,
        visible=visible,
        y=y_dots,
        mode="markers",
        name=name_dots,
        marker=dict(color='red', size=8),
        showlegend=showlegend
    )

In [6]:
def uncertainty_area_scatter(
        visible: bool = True,
        x_lines: np.array = np.array([]),
        y_upper: np.array = np.array([]),
        y_lower: np.array = np.array([]),
        name: str = "mean plus/minus standard deviation",
) -> go.Scatter:

    return go.Scatter(
        visible=visible,
        x=np.concatenate((x_lines, x_lines[::-1])),  # x, then x reversed
        # upper, then lower reversed
        y=np.concatenate((y_upper, y_lower[::-1])),
        fill='toself',
        fillcolor='rgba(189,195,199,0.5)',
        line=dict(color='rgba(200,200,200,0)'),
        hoverinfo="skip",
        showlegend=True,
        name= name,
    )

In [7]:
def add_slider_GPR(figure: go.Figure, parameters):
    figure.data[0].visible = True
    figure.data[1].visible = True

    # Create and add slider
    steps = []
    for i in range(int((len(figure.data) - 1) / 2)):
        step = dict(
            method="update",
            label=f'{parameters[i]: .2f}',
            args=[{
                "visible": [False] * (len(figure.data) - 1) + [True]
            }],
        )
        step["args"][0]["visible"][2 *
                                   i] = True  # Toggle i'th trace to "visible"
        step["args"][0]["visible"][2 * i + 1] = True
        steps.append(step)

    sliders = [dict(
        active=0,
        pad={"t": 50},
        steps=steps,
    )]
    figure.update_layout(sliders=sliders, )
    return figure

In [8]:
def add_slider_to_function(figure:go.Figure, parameters):
    figure.data[0].visible = True

    # Create and add slider
    steps = []
    for i in range(len(figure.data)):
        step = dict(
            method="update",
            label=f'{parameters[i]: .2f}',
            args=[{
                "visible": [False] *len(figure.data) 
            }],
        )
        step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
        steps.append(step)

    sliders = [dict(
        active=0,
        pad={"t": 50},
        steps=steps,
    )]
    figure.update_layout(sliders=sliders, )
    return figure

# Implementation of GPR with squared exponential kernel 

In order to define a gaussian process regressor (GPR) we need a covariance function (also called kernel). The choice of this function will determine the 'shape' of the later GPR.

$$ k(x_1,x_2):= \sigma^2*\exp(-\|x_1-x_2\|^2_2)/(2*l^2))$$
with $$l>0$$ the lengthscale and $$\sigma^2>0$$ the signal variance. 
Lengthscale 
𝑙
l: Determines the smoothness of the predicted function. A small 
𝑙
l makes the model more sensitive to small changes in input, while a large 
𝑙
l results in smoother predictions.
Signal Variance 
𝜎
2
σ 
2
 : Controls the overall amplitude of the function. A larger 
𝜎
2
σ 
2
  leads to more variability in the function’s values.

# GPR CLASS

In [9]:
class SquaredExponentialKernel: 
    def __init__(self, length=1, sigma_f=1):
        self.length = length 
        self.sigma_f = sigma_f

    def __call__(self, argument_1, argument_2):
        return float(self.sigma_f*np.exp(-np.linalg.norm(argument_1-argument_2)**2/(2*self.length**2)))
    

The covariance function defines the relationships between data points in a Gaussian Process and determines how similar or correlated points are.
The squared exponential kernel is a popular covariance function that leads to smooth functions, where points that are close together in the input space are more highly correlated.
The lengthscale 
𝑙
l and signal variance 
𝜎
2
σ 
2
  control the smoothness and variability of the function learned by the Gaussian Process.

In [10]:
x_lines = np.arange(-10, 10, 0.1)
kernel = SquaredExponentialKernel(length=1)

fig0 = go.FigureWidget(data=[
    line_scatter(
        x_lines=x_lines,
        y_lines=np.array([kernel(x, 0) for x in x_lines]),
    )
])

fig0 = update_layout_of_graph(fig0, title='Squared exponential kernel')


@interact(length=(0.1, 3, 0.1), argument_2=(-10, 10, 0.1))
def update(length=1, argument_2=0):
    with fig0.batch_update():
        kernel = SquaredExponentialKernel(length=length)
        fig0.data[0].y = np.array([kernel(x, argument_2) for x in x_lines])


fig0

interactive(children=(FloatSlider(value=1.0, description='length', max=3.0, min=0.1), FloatSlider(value=0.0, d…

FigureWidget({
    'data': [{'line': {'color': 'blue', 'width': 2},
              'name': 'Predicted function',
              'showlegend': True,
              'type': 'scatter',
              'uid': '073fd673-c551-4c75-96f2-276f89320f6c',
              'visible': True,
              'x': array([-1.00000000e+01, -9.90000000e+00, -9.80000000e+00, -9.70000000e+00,
                          -9.60000000e+00, -9.50000000e+00, -9.40000000e+00, -9.30000000e+00,
                          -9.20000000e+00, -9.10000000e+00, -9.00000000e+00, -8.90000000e+00,
                          -8.80000000e+00, -8.70000000e+00, -8.60000000e+00, -8.50000000e+00,
                          -8.40000000e+00, -8.30000000e+00, -8.20000000e+00, -8.10000000e+00,
                          -8.00000000e+00, -7.90000000e+00, -7.80000000e+00, -7.70000000e+00,
                          -7.60000000e+00, -7.50000000e+00, -7.40000000e+00, -7.30000000e+00,
                          -7.20000000e+00, -7.10000000e+00, -7.00000000

In [ ]:
def covariance_matrix(x1, x2, covariance_function):
    return np.array([[covariance_function(a, b) for a in x1] for b in x2])

 Let us shortly recall the formula:
Given training points x1​,...,xn​∈Rm with values y1​,...,yn​∈R, y=(yi​)∈Rn with noise in each point $\mathcal{N}{0,\sigma}$  and points $x{n+1},...,x_k\in \mathbb{R}^m$ for which we want to predict the output, adapting our probability distribution leads to:$$\mathcal{N}(K_K^{-1}y,K_{**}-K_K^{-1}K_^T)$$
with 
$$K= (k(x_i,x_j))_{i,j\leq n}+\sigma^2\mathbb{1}n$$
$$K*= (k(x_i,x_j)){n+1\leq i, j\leq n}$$
$$K{**}= (k(x_i,x_j))_{n+1\leq i,j}$$


Covariance Matrix 𝐾
K:This is a matrix that tells us how related the training data points are to each other. It is built using a kernel function, which measures similarity between points.
Adding noise: We also add noise to this matrix because real-world data has some randomness or measurement error.

Prediction Covariance 
𝐾
∗
∗
K 
∗∗
​
 :

This part measures how related the new data points (where we want to predict) are to each other.
Cross-Covariance 
𝐾
∗
𝐾
K 
∗K
​
 :

This part measures how related the new points are to the training points. It helps us figure out how much the new data should depend on the training data.

In [14]:
class GPR:
    def __init__(self, data_x, data_y, covariance_function = SquaredExponentialKernel(), noise):
        self.data_x = data_x 
        self.data_y = data_y 
        self.covariance_function = covariance_function 
        self.noise = noise 
        self._memory = None  

        self.inv_cov_matrix_of_input_data = np.linalg(covariance_matrix(data_x, data_x, covariance_function)*(noise*3e-7)*np.identity(len(data_x)))

    def predict(self,at_values): 
        k_lower_star = covariance_matrix(self.data_x, at_values, self.covariance_function) #covarianc ematrix ebtween new points and training data
        k_lower_star = covariance_matrix(at_values, at_values, self.covariance_function)
        
        mean_at_values = np.dot(k_lower_star, np.dot(self.data_y, self.inv_cov_matrix_of_input_data.T).T).flatten()
        
        adapted_covar_matrix = k_lower_star2 - np.dot(k_lower_star, np.dot(self.inv_cov_matrix_of_input_data, k_lower_star.T))

        variance = np.diag(cover_matrix)

        self.memory = {'mean': mean_at_values, 'covariance_matrix': cover_matrix, 'variance':variance}

        return mean_at_values



    


SyntaxError: non-default argument follows default argument (3830507152.py, line 2)